In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt

Sort Base Options and Labels

In [ ]:
model_path = "D:\Documents\Education\Bournemouth\Semester2\Masterclass\Code\models\selfie_multiclass_256x256.tflite"
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.ImageSegmenterOptions(base_options=base_options,
                                       output_category_mask=True, output_confidence_masks=False, running_mode=vision.RunningMode.VIDEO)

# Define category labels
labels = {
    "Background": 0,
    "Hair": 1,
    "Body Skin": 2,
    "Face Skin": 3,
    "Clothes": 4,
    "Others (Accessories)": 5
}

# Select a label
selected_label = "Face Skin"
selected_index = labels[selected_label]

# Load video file
vid = cv2.VideoCapture("D:\Documents\Education\Bournemouth\Semester2\Masterclass\Code\data\\videos\smile_1.mp4")

# Get video properties
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
fps = int(vid.get(cv2.CAP_PROP_FPS))

# Define video writers for saving output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")


# Frame index counter
frame_idx = 0


Create Segmenter

In [ ]:
with vision.ImageSegmenter.create_from_options(options) as segmenter:
    while vid.isOpened():
        ret, frame = vid.read()
        if not ret:
            break       # Exit loop if video ends

        # Convert frame to MediaPipe format
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # Segment
        segmentation_result = segmenter.segment_for_video(input_image, frame_idx)
        frame_idx += 1 # Increment frame index

        # Get category mask
        category_mask = np.array(segmentation_result.category_mask.numpy_view())

        # Create binary mask for selected segment
        binary_mask = (category_mask == selected_index).astype(np.uint8) * 255

        # Apply mask to the frame
        segmented_frame = cv2.bitwise_and(frame, frame, mask=binary_mask)

        # Display the og and segmented video
        cv2.imshow("Original Video", frame)
        cv2.imshow(f"Segmented: {selected_label}", segmented_frame)

        # Press q to quit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

# Release resources
vid.release()
cv2.destroyAllWindows()
